In [ ]:
# === Colab 환경 설정 ===
# 이 셀은 Google Colab에서만 실행됩니다.

import os, sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🌐 Google Colab 환경 감지됨")
    print("필요한 패키지를 설치합니다...")
    
    # 기본 패키지 설치
    !pip install -q numpy matplotlib otter-grader
    
    # 저장소 클론 (이미 있으면 건너뜀)
    if not os.path.exists('/content/toy_BBN'):
        !git clone -q https://github.com/cosmo-ojisan/toy_BBN.git /content/toy_BBN
    
    os.chdir('/content/toy_BBN/dist/student')
    sys.path.insert(0, '/content/toy_BBN/dist/student')
    
    # 한글 폰트 설치 (Colab용)
    !apt-get -qq install -y fonts-nanum > /dev/null 2>&1
    
    print("✅ 환경 설정 완료!")
    print(f"현재 작업 위치: {os.getcwd()}")
else:
    print("💻 로컬 환경 감지됨 - 설정을 건너뜁니다.")

# 2차시: 중수소(D) 병목

## 학습 목표

이 노트북을 완료하면 다음을 할 수 있다:

1. **가역 반응** 관점에서 $p+n \rightleftarrows D+\gamma$를 이해한다
2. 왜 온도가 충분히 내려가도 중수소(D)가 **바로 쌓이지 않는지** 설명한다
3. "핵합성 시작 온도" $T_{\text{nuc}}$의 의미를 이해한다
4. **스위치 함수**를 사용한 모형과 1차시 모형을 비교한다

---

## 사전 요구사항

- 1차시 완료 (H-He-4 토이 BBN)
- 동결(freeze-out), 중성자 붕괴, $Y_p = 2X_n$ 개념 이해

In [ ]:
# === 환경 설정 ===
# Google Colab에서 실행할 경우 아래 주석을 해제하세요
# !pip install -q otter-grader

import math
import numpy as np
import matplotlib.pyplot as plt
import warnings

# matplotlib 폰트 경고 억제
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')

# 한글 폰트 설정
try:
    import matplotlib.font_manager as fm
    korean_fonts = ['NanumSquare', 'NanumGothic', 'NanumBarunGothic', 
                    'Malgun Gothic', 'AppleGothic', 'UnDotum', 'Noto Sans CJK KR']
    font_found = False
    for font in korean_fonts:
        try:
            fm.findfont(font, fallback_to_default=False)
            plt.rcParams['font.family'] = font
            font_found = True
            break
        except:
            continue
    if not font_found:
        # Colab 환경에서 나눔폰트 설치 시도
        import subprocess
        import sys
        try:
            subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'koreanize-matplotlib'], 
                          capture_output=True, timeout=30)
            import koreanize_matplotlib
        except:
            pass
except:
    pass

plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

print("✅ 환경 설정 완료!")

In [ ]:
# Otter grader 준비
import os, sys

try:
    import otter
except ImportError:
    !pip install -q otter-grader
    import otter

assert os.path.isdir("tests"), "tests 폴더가 없습니다. 저장소를 제대로 클론했는지 확인하세요."
grader = otter.Notebook()

print("✅ 준비 완료: grader.check('qA1') 처럼 즉시채점을 실행할 수 있어요.")

In [ ]:
# Helper: 한국어 채점 출력
import re, os, sys, glob
from io import StringIO
from contextlib import redirect_stdout, redirect_stderr

# 이 레슨의 테스트 접두사
_LESSON_PREFIXES = ['qD', 'qE', 'qF']

def _list_valid_tests():
    """이 레슨의 테스트 파일 목록 반환"""
    tests = []
    for f in glob.glob('tests/q*.py'):
        name = os.path.splitext(os.path.basename(f))[0]
        if any(name.startswith(prefix) for prefix in _LESSON_PREFIXES):
            tests.append(name)
    return sorted(tests)

def _test_vars_from_file(test_name):
    p = os.path.join('tests', test_name + '.py')
    if not os.path.exists(p):
        return set()
    with open(p, 'r', encoding='utf-8') as f:
        txt = f.read()
    return set(re.findall(r"(ans_[A-Za-z0-9_]+)", txt))

# 원본 grader.check
_original_check = grader.check

def _korean_check(test_name, show_output=True):
    """한국어 메시지와 함께 채점"""
    result = _original_check(test_name)
    if show_output:
        if result.passed:
            print(f"✅ {test_name}: 정답!")
        else:
            needed_vars = _test_vars_from_file(test_name)
            g = globals()
            missing = [v for v in needed_vars if v not in g]
            if missing:
                print(f"❌ {test_name}: 변수 누락 → {missing}")
            else:
                print(f"❌ {test_name}: 오답 — 다시 확인해보세요.")
    return result

grader.check = _korean_check

def _korean_check_all():
    """이 레슨의 테스트만 실행"""
    print("📊 전체 채점 시작...\n")
    valid_tests = _list_valid_tests()
    passed = 0
    failed = 0
    for test_name in valid_tests:
        result = _korean_check(test_name, show_output=False)
        if result.passed:
            passed += 1
        else:
            print(f"❌ {test_name}: 오답")
            failed += 1
    print(f"\n📊 전체 결과: {passed}/{passed+failed} 통과")
    if failed == 0:
        print("🎉 모든 문항 정답!")

grader.check_all = _korean_check_all

---

# 1차시 핵심 함수 (독립 실행용)

2차시를 독립적으로 실행하기 위해 1차시의 핵심 함수들을 아래에 포함합니다.
**이 셀은 수정하지 마세요!**

In [ ]:
# === 1차시 핵심 상수 및 함수 ===
# (2차시 독립 실행을 위해 포함)

# 물리 상수
A_default = 1.32        # t = A / T^2 관계의 상수 [초·MeV^2]
gamma0_default = 1.3    # Γ = γ₀ T^5 관계의 상수 [1/초·MeV^5]
Q_default = 1.293       # 중성자-양성자 질량차 [MeV]
tau_n_default = 879.0   # 중성자 평균 수명 [초]

def t_of_T(T, A=A_default):
    """온도 T에서 시간 t 계산 (토이 모델)"""
    return A / (T * T)

def Gamma_of_T(T, gamma0=gamma0_default):
    """온도 T에서 약한 반응률 Γ 계산"""
    return gamma0 * (T ** 5)

def Xn_eq(T, Q=Q_default):
    """평형 중성자 분율"""
    return 1.0 / (1.0 + math.exp(Q / T))

def np_to_Xn(np_ratio):
    """n/p 비를 Xn으로 변환"""
    return np_ratio / (1.0 + np_ratio)

def Xn_to_np(Xn):
    """Xn을 n/p 비로 변환"""
    if Xn >= 1.0:
        return float('inf')
    return Xn / (1.0 - Xn)

def exponential_relaxation(x, x_eq, Gamma, dt):
    """지수 완화 (안정적 수치 적분)"""
    return x_eq + (x - x_eq) * math.exp(-Gamma * dt)

def make_T_grid(T_start=10.0, T_nuc=0.07, n_steps=600):
    """온도 그리드 생성 (로그 스케일)"""
    return np.logspace(np.log10(T_start), np.log10(T_nuc), n_steps)

def step_Xn(Xn, T_prev, T_now, A=A_default, gamma0=gamma0_default, Q=Q_default, tau_n=tau_n_default):
    """한 스텝 업데이트 (1차시 통합 함수)"""
    t_prev = t_of_T(T_prev, A)
    t_now = t_of_T(T_now, A)
    dt = t_now - t_prev
    
    T_mid = math.sqrt(T_prev * T_now)
    lam_np = Gamma_of_T(T_mid, gamma0)
    lam_pn = lam_np * math.exp(-Q / T_mid)
    Gamma_wk = lam_np + lam_pn
    Xeq_mid = lam_pn / Gamma_wk
    
    Xn_temp = exponential_relaxation(Xn, Xeq_mid, Gamma_wk, dt)
    Xn_next = Xn_temp * math.exp(-dt / tau_n)
    Xn_next = max(0.0, min(1.0, Xn_next))
    return Xn_next

def run_bbn_simulation(T_start=10.0, T_nuc=0.07, n_steps=600):
    """1차시 BBN 시뮬레이션 실행"""
    T = make_T_grid(T_start=T_start, T_nuc=T_nuc, n_steps=n_steps)
    Xn = Xn_eq(T[0])
    Xn_list = [Xn]
    t_list = [t_of_T(T[0])]
    Xeq_list = [Xn_eq(T[0])]
    
    for i in range(1, len(T)):
        Xn = step_Xn(Xn, float(T[i-1]), float(T[i]))
        Xn_list.append(Xn)
        t_list.append(t_of_T(T[i]))
        Xeq_list.append(Xn_eq(T[i]))
    
    Yp = 2 * Xn_list[-1]
    return {
        "T": T,
        "t": np.array(t_list),
        "Xn": np.array(Xn_list),
        "Xn_eq": np.array(Xeq_list),
        "Yp": float(Yp),
    }

print("✅ 1차시 핵심 함수 로드 완료!")

---

# Part D: 개념 이해 — 중수소 병목

---

## D0. 1차시 복습 (Warm-up)

1차시에서 배운 핵심 개념을 복습합니다.

### D0.1 동결(Freeze-out)은 언제 일어나는가?

동결은 다음 중 어떤 조건에서 일어나는가?

- (A) 온도가 0이 될 때
- (B) 중성자가 모두 붕괴할 때
- (C) $\Gamma/H$가 1보다 작아질 때
- (D) 양성자와 중성자 수가 같아질 때

In [ ]:
# D0.1: 동결 조건
ans_D0_1 = ...  # "A", "B", "C", "D" 중 선택

In [ ]:
grader.check('qD0', global_env=globals())

### D0.2 중성자 붕괴

중성자 붕괴에 대한 설명으로 옳은 것은?

- (A) 중성자는 갑자기 모두 붕괴한다
- (B) 중성자는 **일정 비율로** 감소한다 (지수 감쇠)
- (C) 중성자는 온도가 낮아지면 붕괴가 멈춘다

In [ ]:
# D0.2: 중성자 붕괴
ans_D0_2 = ...  # "A", "B", "C" 중 선택

### D0.3 헬륨 질량비 Yp

최종 중성자 분율이 $X_n = 0.12$일 때, 헬륨 질량비 $Y_p$는?

**힌트:** $Y_p = 2X_n$

In [ ]:
# D0.3: Yp 계산
Xn_final = 0.12
ans_D0_3 = ...  # Yp 값을 계산하세요

---

## D1. 가역 반응이란?

화학에서 배운 **가역 반응(reversible reaction)**을 떠올려 보자.

### 예시: 물의 증발과 응결

$$\text{물(액체)} \rightleftarrows \text{물(기체)}$$

- **정반응(→):** 물이 증발
- **역반응(←):** 수증기가 응결

### 평형 상태란?

평형은 "반응이 멈춘 것"이 **아니다**!

평형 = **정반응 속도 = 역반응 속도**

양쪽으로 계속 반응하지만, 알짜 변화가 없는 상태.

### D1.1 평형에 대한 이해

평형 상태에 대한 올바른 설명은?

- (A) 반응이 완전히 멈춘 상태
- (B) 정반응과 역반응이 **같은 속도**로 일어나는 상태
- (C) 정반응만 일어나는 상태

In [ ]:
# D1.1: 평형 이해
ans_D1_1 = ...  # "A", "B", "C" 중 선택

In [ ]:
grader.check('qD1', global_env=globals())

---

## D2. 중수소 생성과 광분해

### 핵반응도 가역이다!

$$p + n \rightleftarrows D + \gamma$$

| 방향 | 반응 | 이름 |
|------|------|------|
| 정반응 (→) | $p + n \to D + \gamma$ | 중수소 **생성** |
| 역반응 (←) | $D + \gamma \to p + n$ | 중수소 **광분해** |

> 🔴 **블랙박스:** 중수소 결합에너지 ~2.2 MeV는 핵물리 실험에서 측정된 값이다. 광자 에너지가 이 값보다 크면 D를 깰 수 있다.

### 🔨 "망치 비유"

- **광자($\gamma$)** = 에너지를 가진 "망치"
- **고온** = 망치가 매우 많고 세다 → 결합을 계속 부순다
- **저온** = 망치가 줄고 약해진다 → 결합이 유지된다

### D2.1 고온에서 중수소가 쌓이지 않는 이유

고온에서 중수소(D)가 쌓이지 않는 이유로 가장 적절한 것은?

- (A) 광분해(역반응)가 생성(정반응)보다 **훨씬 빠르기** 때문
- (B) 중수소가 아예 만들어지지 않기 때문
- (C) 중성자가 없기 때문

In [ ]:
# D2.1: 고온에서 D가 안 쌓이는 이유
ans_D2_1 = ...  # "A", "B", "C" 중 선택

In [ ]:
grader.check('qD2', global_env=globals())

---

## D3. 중수소 병목 (Deuterium Bottleneck)

### "병목"이란?

병목(bottleneck) = 좁은 구간 때문에 전체 흐름이 막히는 현상

![병목 비유](https://upload.wikimedia.org/wikipedia/commons/thumb/5/51/Bottleneck.svg/200px-Bottleneck.svg.png)

### BBN에서의 병목

1. 고온에서는 D가 만들어져도 **바로 깨진다** (광분해)
2. 온도가 내려가야 D가 **살아남기 시작**한다
3. D가 쌓여야 → He-3, T → **He-4**로 갈 수 있다

**즉, D가 쌓이지 않으면 핵합성이 진행되지 않는다!**

이것이 바로 **중수소 병목**이다.

### D3.1 병목 이해 확인

다음 빈칸을 채우세요:

> "중수소가 쌓여야 핵합성이 시작되므로, 중수소가 살아남지 못하는 구간을 중수소 _____ 이라고 한다." 

In [ ]:
# D3.1: 빈칸 채우기
ans_D3_1 = ...  # "병목" 또는 "bottleneck" 입력

In [ ]:
grader.check('qD3', global_env=globals())

---

## D4. 스위치 함수 — 핵합성 "시작 버튼"

### 토이 모델에서의 근사

정밀한 핵반응 계산 대신, **스위치 함수**로 단순화할 수 있다:

$$B(T) = \begin{cases} 0 & (T > T_D) \\ 1 & (T \le T_D) \end{cases}$$

- $B(T) = 0$: 핵합성 **꺼짐** (D가 살아남지 못함)
- $B(T) = 1$: 핵합성 **켜짐** (D가 쌓이기 시작)
- $T_D$: 병목이 열리는 온도 (~0.07 MeV)

> 🔴 **블랙박스:** 스위치 함수는 복잡한 **사하 방정식(Saha equation)**의 결과를 단순화한 것이다. 정밀한 유도는 통계역학에서 다루며, 여기서는 결과만 사용한다.

### 더 부드러운 스위치: 시그모이드

$$B(T) = \frac{1}{1 + \exp\left(\frac{T - T_D}{\Delta}\right)}$$

- $\Delta$가 작으면: 급격한 전환 (계단 함수에 가까움)
- $\Delta$가 크면: 부드러운 전환

### D4.1 스위치 함수의 의미

스위치 함수 $B(T)$가 0에서 1로 바뀌는 것은 무엇을 의미하는가?

- (A) 중성자가 모두 붕괴함
- (B) 핵합성이 **시작됨** (D가 살아남기 시작)
- (C) 우주가 팽창을 멈춤

In [ ]:
# D4.1: 스위치 함수 의미
ans_D4_1 = ...  # "A", "B", "C" 중 선택

In [ ]:
grader.check('qD4', global_env=globals())

---

## D5. 핵합성 시작 온도 $T_{\text{nuc}}$

### 1차시에서의 가정

1차시에서는 $T_{\text{nuc}} = 0.07$ MeV를 **고정값**으로 사용했다.

### 병목 관점에서의 의미

$T_{\text{nuc}}$는 사실 **"중수소 병목이 열리는 온도"**와 같다!

- 병목이 열림 = D가 살아남음 = 핵합성 시작 가능

### 두 가지 모형

| 모형 | $T_{\text{nuc}}$ 결정 방법 | 장점 |
|------|---------------------------|------|
| A (1차시) | 고정값 사용 (0.07 MeV) | 단순 |
| B (2차시) | 스위치 함수로 자동 결정 | 물리적 의미 명확 |

### D5.1 T_nuc의 물리적 의미

$T_{\text{nuc}}$의 물리적 의미로 가장 적절한 것은?

- (A) 핵합성이 **시작되는** 온도 (병목이 열림)
- (B) 핵합성이 **끝나는** 온도
- (C) 중성자가 모두 **붕괴하는** 온도

In [ ]:
# D5.1: T_nuc 의미
ans_D5_1 = ...  # "A", "B", "C" 중 선택

In [ ]:
grader.check('qD5', global_env=globals())

---

# Part E: 미니 코딩 — 스위치 함수 구현

---

## E0. 계단 스위치 함수

가장 단순한 스위치: 계단 함수

$$B(T) = \begin{cases} 0 & (T > T_D) \\ 1 & (T \le T_D) \end{cases}$$

In [ ]:
# E0: 계단 스위치 함수
def step_switch(T, T_D=0.07):
    """
    계단 스위치 함수
    
    Parameters:
        T: 온도 [MeV]
        T_D: 병목이 열리는 온도 [MeV]
    
    Returns:
        0.0 (T > T_D) 또는 1.0 (T <= T_D)
    """
    # ★빈칸★: T가 T_D 이하일 때 1.0, 아니면 0.0 반환
    # 힌트: if-else 또는 삼항 연산자 사용
    return ...

# 테스트
print("step_switch(0.1) =", step_switch(0.1))   # 0.1 > 0.07 → 0.0
print("step_switch(0.05) =", step_switch(0.05)) # 0.05 < 0.07 → 1.0

In [ ]:
grader.check('qE0', global_env=globals())

---

## E1. 시그모이드 스위치 함수

더 부드러운 전환을 위한 시그모이드:

$$B(T) = \frac{1}{1 + \exp\left(\frac{T - T_D}{\Delta}\right)}$$

In [ ]:
# E1: 시그모이드 스위치 함수
def sigmoid_switch(T, T_D=0.07, delta=0.01):
    """
    시그모이드 스위치 함수
    
    Parameters:
        T: 온도 [MeV]
        T_D: 병목이 열리는 온도 [MeV]
        delta: 전환 폭 [MeV] (작을수록 급격)
    
    Returns:
        0~1 사이 값
    """
    # ★빈칸★: 시그모이드 공식 구현
    # 힌트: 1 / (1 + math.exp((T - T_D) / delta))
    return ...

# 테스트
print("sigmoid_switch(0.1) =", round(sigmoid_switch(0.1), 4))   # ~0.05
print("sigmoid_switch(0.07) =", round(sigmoid_switch(0.07), 4)) # 0.5
print("sigmoid_switch(0.05) =", round(sigmoid_switch(0.05), 4)) # ~0.88

In [ ]:
grader.check('qE1', global_env=globals())

---

## E2. $T_{\text{nuc}}$ 자동 탐색

스위치 함수가 0.5가 되는 온도를 찾아 $T_{\text{nuc}}$로 사용한다.

**제공 함수:** `find_T_nuc_from_switch()`

In [ ]:
# === 제공 함수 ===
def find_T_nuc_from_switch(switch_func, T_start=1.0, T_end=0.01, tolerance=0.001):
    """스위치가 0.5가 되는 온도를 이진 탐색으로 찾음"""
    T_low, T_high = T_end, T_start
    while T_high - T_low > tolerance:
        T_mid = (T_low + T_high) / 2
        if switch_func(T_mid) < 0.5:
            T_high = T_mid
        else:
            T_low = T_mid
    return (T_low + T_high) / 2

# E2: 자동 탐색 실행
# ★빈칸★: sigmoid_switch 함수를 사용해서 T_nuc_auto를 찾으세요
# 힌트: find_T_nuc_from_switch(sigmoid_switch)
T_nuc_auto = ...

print(f"자동으로 찾은 T_nuc = {T_nuc_auto:.4f} MeV")

In [ ]:
grader.check('qE2', global_env=globals())

---

## E3. 스위치 함수 그래프

두 가지 스위치 함수를 그래프로 비교해 봅시다.

In [ ]:
# E3: 스위치 함수 그래프
T_range = np.linspace(0.01, 0.2, 200)

# ★빈칸★: 두 스위치 함수 값을 계산하세요
# 힌트: [step_switch(T) for T in T_range]
B_step = ...      # 계단 스위치
B_sigmoid = ...   # 시그모이드 스위치

# 그래프 그리기
plt.figure(figsize=(8, 5))
plt.plot(T_range, B_step, 'b-', linewidth=2, label='계단 스위치')
plt.plot(T_range, B_sigmoid, 'r--', linewidth=2, label='시그모이드 스위치')
plt.axvline(x=0.07, color='gray', linestyle=':', label='T_D = 0.07 MeV')
plt.axhline(y=0.5, color='green', linestyle=':', alpha=0.5)
plt.xlabel('온도 T [MeV]')
plt.ylabel('스위치 값 B(T)')
plt.title('병목 스위치 함수 비교')
plt.legend()
plt.grid(True, alpha=0.3)
plt.gca().invert_xaxis()  # 온도 감소 방향
plt.show()

In [ ]:
grader.check('qE3', global_env=globals())

---

# Part F: 모형 비교 — 고정 vs 자동 $T_{\text{nuc}}$

---

## F0. 모형 A: 고정 $T_{\text{nuc}}$ (1차시 방식)

1차시에서 사용한 방법: $T_{\text{nuc}} = 0.07$ MeV 고정

In [ ]:
# F0: 모형 A 실행 (고정 T_nuc)
print("=== 모형 A: 고정 T_nuc = 0.07 MeV ===")
result_A = run_bbn_simulation(T_nuc=0.07)
print(f"Yp (모형 A) = {result_A['Yp']:.4f}")

---

## F1. 모형 B: 자동 $T_{\text{nuc}}$ (스위치 기반)

스위치 함수에서 자동으로 찾은 $T_{\text{nuc}}$ 사용

In [ ]:
# F1: 모형 B 실행 (자동 T_nuc)
print("=== 모형 B: 자동 T_nuc (스위치 기반) ===")

# ★빈칸★: T_nuc_auto를 사용해서 시뮬레이션 실행
# 힌트: run_bbn_simulation(T_nuc=T_nuc_auto)
result_B = ...

print(f"T_nuc (자동) = {T_nuc_auto:.4f} MeV")
print(f"Yp (모형 B) = {result_B['Yp']:.4f}")

In [ ]:
grader.check('qF1', global_env=globals())

---

## F2. 두 모형 비교 그래프

In [ ]:
# F2: 두 모형 비교 그래프
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 그래프 1: Xn vs t
ax1 = axes[0]
ax1.plot(result_A["t"], result_A["Xn"], 'b-', linewidth=2, label='모형 A (고정)')
ax1.plot(result_B["t"], result_B["Xn"], 'r--', linewidth=2, label='모형 B (자동)')
ax1.plot(result_A["t"], result_A["Xn_eq"], 'k:', alpha=0.5, label='평형')
ax1.set_xscale("log")
ax1.set_xlabel("시간 t [초]")
ax1.set_ylabel("Xn")
ax1.set_title("Xn(t) 비교")
ax1.legend()
ax1.grid(True, alpha=0.3)

# 그래프 2: n/p vs T
ax2 = axes[1]
eps = 1e-12
np_A = np.maximum(eps, result_A["Xn"]) / np.maximum(eps, 1 - result_A["Xn"])
np_B = np.maximum(eps, result_B["Xn"]) / np.maximum(eps, 1 - result_B["Xn"])
ax2.plot(result_A["T"], np_A, 'b-', linewidth=2, label='모형 A')
ax2.plot(result_B["T"], np_B, 'r--', linewidth=2, label='모형 B')
ax2.set_xscale("log")
ax2.set_yscale("log")
ax2.invert_xaxis()
ax2.set_xlabel("온도 T [MeV]")
ax2.set_ylabel("n/p")
ax2.set_title("n/p(T) 비교")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 결과 비교:")
print(f"   모형 A (고정): Yp = {result_A['Yp']:.4f}")
print(f"   모형 B (자동): Yp = {result_B['Yp']:.4f}")
print(f"   차이: {abs(result_A['Yp'] - result_B['Yp']):.4f}")

### F2.1 그래프 해석

병목이 **늦게 열리면** (T_nuc이 더 낮으면) $Y_p$는 어떻게 되는가?

- (A) 증가한다
- (B) 감소한다
- (C) 변하지 않는다

**힌트:** 병목이 늦게 열리면 → 중성자가 더 오래 붕괴 → X_n 감소 → Yp = 2X_n 감소

In [ ]:
# F2.1: 병목 지연 효과
ans_F2_1 = ...  # "A", "B", "C" 중 선택

### F2.2 병목의 역할

중수소 병목의 주된 역할은?

- (A) 중성자 붕괴를 멈춘다
- (B) 핵합성 **시작 시점**을 결정한다
- (C) 약한 상호작용을 빠르게 만든다

In [ ]:
# F2.2: 병목의 역할
ans_F2_2 = ...  # "A", "B", "C" 중 선택

In [ ]:
grader.check('qF2', global_env=globals())

---

## F3. Exit Ticket — 최종 이해도 점검

### F3.1 OX 문제

"중수소 병목은 '중수소가 아예 만들어지지 않는 현상'이다."

- 참(True) / 거짓(False)?

In [ ]:
# F3.1: OX 문제
ans_F3_1 = ...  # True 또는 False

### F3.2 단답형

병목이 늦게 열리면 중성자가 더 오래 붕괴한다. 그러면 $Y_p$는 (증가/감소)한다.

In [ ]:
# F3.2: 단답형
ans_F3_2 = ...  # "증가" 또는 "감소"

In [ ]:
grader.check('qF3', global_env=globals())

---

# ⚠️ 오해하기 쉬운 개념

## 오해 1: "병목 = D가 아예 안 만들어짐"

**정정:** D는 고온에서도 만들어진다! 다만 **바로 깨질 뿐**이다.
병목은 "만들어지지 않음"이 아니라 "살아남지 못함"을 의미한다.

## 오해 2: "온도만 낮아지면 즉시 결합한다"

**정정:** 정반응과 역반응이 **경쟁**한다. 어느 쪽이 더 우세한지가 핵심이다.
온도가 낮아져도 역반응(광분해)이 아직 우세하면 D는 쌓이지 않는다.

## 오해 3: "스위치는 임의 설정이다"

**정정:** 스위치는 복잡한 물리(광분해 경쟁)를 **압축**한 것이다.
압축의 이유(왜 그 온도에서 열리는가)가 분명해야 정당하게 사용할 수 있다.

---

# 🎯 2차시 핵심 정리

## 세 가지 큰 질문과 답

### Q1. 왜 온도가 충분히 내려가도 중수소(D)가 바로 쌓이지 않는가?

**A:** 고온에서는 **광분해(역반응)**가 우세해서, 중수소가 만들어져도 바로 깨진다.
온도가 충분히 내려가야 광분해가 약해지고, 그제서야 중수소가 "살아남기" 시작한다.

### Q2. "가역 반응" 관점에서 $p+n \rightleftarrows D+\gamma$를 어떻게 이해할 수 있는가?

**A:** 정반응(생성)과 역반응(광분해)이 **경쟁**한다.
고온에서는 역반응이 우세 → D 쌓이지 않음
저온에서는 정반응이 우세 → D 쌓임 시작

### Q3. '핵합성 시작 온도' $T_{\text{nuc}}$는 무엇을 의미하며, 1차시의 가정과 어떤 관계가 있는가?

**A:** $T_{\text{nuc}}$는 **병목이 열리는 온도**다.
1차시에서는 이를 고정값(0.07 MeV)으로 가정했지만,
실제로는 스위치 함수를 통해 물리적으로 결정된다.

---

## 다음 차시 예고

2차시는 **"언제 시작되는가"**를 다루었습니다.

3차시에서는 실제로 D, T, He-3가 어떻게 He-4로 흘러가는지,
**"반응 네트워크"**를 최소 형태로 구현합니다!

---

축하합니다! 🎉 2차시를 완료했습니다!

In [ ]:
# 전체 채점
grader.check_all()